## 1. urllib 활용

In [1]:
import requests as r
import urllib.request as ur
import os
import pandas as pd
import time

In [2]:
# 해당 디렉토리가 없으면 만드는 함수
def createFolder(dir_name):
    try:
        if not os.path.exists(dir_name):
            os.makedirs(dir_name)
    except OSError:
        print ('Error: Creating directory. ' +  dir_name)

In [3]:
# 쿼리를 요청할 검색어 리스트 만들거나 불러오기
name_list = ['Barak Obama','Donald Trump' ,'Joe Biden']

In [4]:
key1 = "" # Your bing API key

In [22]:
def make_img_list(lst, apikey):
  # 시작 시간 저장
  start = time.time()
  # 에러 로그 저장을 위한 리스트 만들기
  error_log = []
  # 주어진 리스트에 있는 모든 이름 검색
  for item in lst:

    # 폴더 만들고 경로 이동하기
    os.chdir("./bing_img") # your designated directory
    createFolder(item)
    os.chdir(f"./{item}")

    url = "https://api.bing.microsoft.com/v7.0/images/search"
    headers = {
              "Ocp-Apim-Subscription-Key" : key1
              }
    param = {"q" : item,
          "count" : 10,
          "imageType" : "Photo",
          "imageContent" : "Face",
          "minHeight" : 256,
          "safeSearch" : "strict",
          "color" : "ColorOnly"
          }
    s = r.Session()
    test = s.get(url,headers=headers, params=param).json()
    # 검색결과 링크에서 이미지 저장하기

    for i in range(0,len(test.get("value"))):
        print(item,'-',i)
        print(test.get('value')[i].get('contentUrl'))
        file_name = f"{item}_{i}.{test.get('value')[i].get('encodingFormat')}"
        try:
          ur.urlretrieve(test.get('value')[i].get('contentUrl'),file_name)
        except Exception as e: 
          print(e)
          error_log.append(f"{file_name}: {e}")

          pass
    os.chdir("../..")
  time_taken = time.time()-start
  return time_taken, error_log

In [23]:
make_img_list(name_list, key1)

Barak Obama - 0
https://media.vanityfair.com/photos/5eb6e24a77a607fc7f7f1572/master/w_2560%2Cc_limit/Barack-Obama.jpg
Barak Obama - 1
https://issuepedia.org/wikiup/thumb/f/f8/Officialportrait.jpg/1200px-Officialportrait.jpg
Barak Obama - 2
https://images-na.ssl-images-amazon.com/images/I/B1iW0a3boMS.jpg
Barak Obama - 3
https://www.famousbirthsdeaths.com/wp-content/uploads/2015/12/is-barack-obama-dead-or-alive.jpg
HTTP Error 403: Forbidden
Barak Obama - 4
https://www.nbforum.com/wp-content/uploads/obama-600x587.jpg
Barak Obama - 5
https://www.espreso.co.rs/data/images/2020/07/17/18/790599_barak-obama_ls.jpg
Barak Obama - 6
http://abovethelaw.com/wp-content/uploads/2015/04/barack-obama.jpg
HTTP Error 403: Forbidden
Barak Obama - 7
https://actorsareidiots.com/wp-content/uploads/2020/06/BarackObama.jpg
HTTP Error 403: Forbidden
Barak Obama - 8
http://tmillan1.files.wordpress.com/2012/03/obamaheadshot.jpg
Barak Obama - 9
https://images.newrepublic.com/8822072d3da7d7042a4e50a59273438c03842ba

(28.09617280960083,
 ['Barak Obama_3.jpeg: HTTP Error 403: Forbidden',
  'Barak Obama_6.jpeg: HTTP Error 403: Forbidden',
  'Barak Obama_7.jpeg: HTTP Error 403: Forbidden',
  'Barak Obama_9.jpeg: HTTP Error 403: Forbidden',
  'Donald Trump_0.jpeg: HTTP Error 403: Forbidden',
  'Donald Trump_1.jpeg: HTTP Error 403: Forbidden',
  'Donald Trump_3.jpeg: HTTP Error 403: Forbidden',
  'Donald Trump_6.jpeg: HTTP Error 403: Forbidden',
  'Donald Trump_9.jpeg: HTTP Error 403: Forbidden',
  'Joe Biden_8.jpeg: HTTP Error 403: Forbidden',
  'Joe Biden_9.jpeg: HTTP Error 406: Not Acceptable'])